In [ ]:
from pathlib import Path
from datetime import datetime, timedelta
import geopandas as gpd

from ipyfilechooser import FileChooser

import opensarlab_lib as asfn
import asf_search as asf
import hyp3_sdk as sdk

### Select the chip shape file

In [ ]:
fc = FileChooser(Path.cwd())
display(fc)

### Load the chip geometry from the chip shapefile

In [ ]:
shape_path = Path(fc.selected)
shape_path

In [ ]:
geometry_gdf = gpd.read_file(shape_path)
geometry = geometry_gdf['geometry'][0]
geometry

### Collect the chip directory and ID

In [ ]:
chip_dir = shape_path.parent
chip_id = chip_dir.name.split('_')[-1]
chip_dir

### Determine a date range containing the most recent 12-day Sentinel-1A repeat cycle

In [ ]:
current_time = datetime.now()
current_time_str = current_time.strftime('%Y-%m-%dT%H:%M:%SZ')

twelve_days_ago = current_time  - timedelta(days=12)
twelve_days_ago_str = twelve_days_ago.strftime('%Y-%m-%dT%H:%M:%SZ')

### Select a flight direction

In [ ]:
flight_dir = asfn.select_parameter(['ASCENDING', 'DESCENDING'])
display(flight_dir)

### Search Sentinel-1A SLCs acquisitions intersecting the chip within the past 12 days, in the chosen flight direction

In [ ]:
opts = {
    'platform': asf.PLATFORM.SENTINEL1A,
    'start': twelve_days_ago_str,
    'end': current_time_str,
    'processingLevel': 'SLC',
    'flightDirection': flight_dir.value,
}

scenes = asf.geo_search(geometry.wkt, **opts)
scenes

### Organize scenes in a dictionary by acquisition date and select a desired date if more than 1 are present

In [ ]:
scenes_by_date = dict()
for scene in scenes:
    date_str = datetime.strptime(scene.properties['startTime'], '%Y-%m-%dT%H:%M:%S.%fZ').date().strftime('%Y-%m-%d')
    if date_str not in scenes_by_date:
        scenes_by_date[date_str] = [scene]
    else:
        scenes_by_date[date_str].append(scene)

In [ ]:
if len(scenes_by_date) > 1:
    print("Select a scene acquisition date")
    date_options = asfn.select_parameter([d for d in scenes_by_date])
    display(date_options)

In [ ]:
if len(scenes_by_date) > 1:
    key = date_options.value
else:
    key = list(scenes_by_date.keys())[0]
print(f'We will order {len(scenes_by_date[key])} HyP3 RTCs from Sentinel-1A scenes intersecting chip {chip_id} and acquired on {key}')

### Authenticate on HyP3 with EarthData credentials and order RTCs for the scenes on the chosen acquisition date

In [ ]:
hyp3 = sdk.HyP3(prompt=True)

In [ ]:
hyp3_project_name = f"S1A_SLC_{key}_chip_{chip_id}"
scene_names = [s.properties['sceneName'] for s in scenes_by_date[key]]

rtc_jobs = sdk.Batch()
for s in scene_names:
    rtc_jobs += hyp3.submit_rtc_job(s, name=hyp3_project_name)
print(rtc_jobs)